# Customer Support Bot Project Template

Welcome to the Customer Support Bot project! This notebook will help you build "Chip", a friendly AI customer support assistant for the fictional company "GadgetCo" that helps customers with their Smart-Widget device.

## Learning Objectives

By completing this project, you will learn:
- How to create detailed system prompts for AI agents
- How to use multiple MCP servers in a single agent
- How to design conditional tool usage logic
- How to work with local file storage for knowledge bases
- How to integrate external search capabilities

## Project Overview

You'll build "Chip", a customer support agent that:
1. First checks internal company documents (FAQ and manual)
2. Falls back to external search if needed
3. Maintains a friendly, professional persona
4. Never guesses - escalates to humans when uncertain

This notebook will set up the knowledge base files and explain the approach. You'll implement the agent yourself!

## Section 1: Setup

First, let's install the Aurite package and configure our environment.

In [1]:
# Install the Aurite package
%pip install /home/wilcoxr/workspace/aurite/aurite-agents/dist/aurite-0.3.17-py3-none-any.whl --force-reinstall

Processing /home/wilcoxr/workspace/aurite/aurite-agents/dist/aurite-0.3.17-py3-none-any.whl
  Using cached mcp-1.9.4-py3-none-any.whl.metadata (28 kB)
  Using cached httpx-0.28.1-py3-none-any.whl.metadata (7.1 kB)
  Using cached anyio-4.9.0-py3-none-any.whl.metadata (4.7 kB)
  Using cached igittigitt-2.1.5-py3-none-any.whl.metadata (22 kB)
  Using cached structlog-25.4.0-py3-none-any.whl.metadata (7.6 kB)
  Using cached sqlalchemy-2.0.41-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (9.6 kB)
  Using cached anthropic-0.54.0-py3-none-any.whl.metadata (25 kB)
  Using cached annotated_types-0.7.0-py3-none-any.whl.metadata (15 kB)
  Using cached certifi-2025.6.15-py3-none-any.whl.metadata (2.4 kB)
  Using cached cffi-1.17.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached click-8.2.1-py3-none-any.whl.metadata (2.5 kB)
  Using cached cryptography-43.0.3-cp39-abi3-manylinux_2_28_x86_64.whl.metadata (5.4 kB)
  Using cached distro

In [2]:
# Configure API Keys
import os
from getpass import getpass

try:
  from google.colab import userdata #type: ignore
  # Fetch the secret value from Colab's secret manager and set it as an environment variable so your script can find it
  os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')
  os.environ["SMITHERY_API_KEY"] = userdata.get('SMITHERY_API_KEY')
  os.environ["SMITHERY_PROFILE_ID"] = userdata.get('SMITHERY_PROFILE_ID')
except ImportError: # This is not a real error, just a check if running in Colab
    from dotenv import load_dotenv
    load_dotenv()  # Load environment variables from a .env file if it exists
except Exception as e:
    print(f"Error fetching from Colab's secret manager: {e}")

if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass("Enter your OpenAI API key: ")
if "SMITHERY_API_KEY" not in os.environ:
    os.environ["SMITHERY_API_KEY"] = getpass("Enter your Smithery API key: ")
if "SMITHERY_PROFILE_ID" not in os.environ:
    os.environ["SMITHERY_PROFILE_ID"] = getpass("Enter your Smithery Profile ID: ")

# Set up a dedicated directory for our knowledge base files
# and configure the file storage server to use it.
from pathlib import Path

# Create a dedicated directory for the knowledge base
docs_dir = Path("customer_support_docs")
docs_dir.mkdir(exist_ok=True)

# Set the environment variable to this new directory.
# The local_file_storage server will use this path as its root.
os.environ['AURITE_PROJECT_DIR'] = str(docs_dir.resolve())

print(f"✅ Set AURITE_PROJECT_DIR to: {os.environ['AURITE_PROJECT_DIR']}")

✅ Set AURITE_PROJECT_DIR to: /home/wilcoxr/workspace/aurite/aurite-agents/docs/notebooks/customer_support_docs


In [3]:
# Helper function for displaying agent responses
from IPython.display import display, Markdown

def display_agent_response(agent_name: str, query: str, response: str):
  """Formats and displays the agent's response in a structured Markdown block."""

  output = f"""
  <div style="border: 1px solid #D1D5DB; border-radius: 8px; margin-top: 20px; font-family: sans-serif; box-shadow: 0 4px 6px rgba(0,0,0,0.05);">
    <div style="background-color: #F3F4F6; padding: 10px 15px; border-bottom: 1px solid #D1D5DB; border-radius: 8px 8px 0 0;">
      <h3 style="margin: 0; font-size: 16px; color: #1F2937; display: flex; align-items: center;">
        <span style="margin-right: 8px;">🤖</span>
        Agent Response: <code style="background-color: #E5E7EB; color: #374151; padding: 2px 6px; border-radius: 4px; margin-left: 8px;">{agent_name}</code>
      </h3>
    </div>
    <div style="padding: 15px;">
      <p style="margin: 0 0 10px 0; color: #6B7280; font-size: 14px;">
        <strong>Your Query:</strong>
      </p>
      <p style="background-color: #F9FAFB; margin: 0 0 15px 0; color: #1F2937; border: 1px solid #E5E7EB; border-left: 3px solid #9CA3AF; padding: 10px 12px; border-radius: 4px;">
        <em>\"{query}\"</em>
      </p>
      <hr style="border: none; border-top: 1px dashed #D1D5DB; margin-bottom: 15px;">
      <p style="margin: 0 0 10px 0; color: #6B7280; font-size: 14px;">
        <strong>Result:</strong>
      </p>
      <div style="background-color: #FFFFFF; padding: 15px; border-radius: 5px; border: 1px solid #E5E7EB; color: #1F2937; font-size: 15px; line-height: 1.6;">
        {response}
      </div>
    </div>
  </div>
  """
  display(Markdown(output))

In [4]:
# Initialize Aurite
from aurite import Aurite

aurite = Aurite()
await aurite.initialize()
print("✅ Aurite initialized successfully!")

INFO     [aurite.config.component_manager] User project config directory not found at /home/wilcoxr/workspace/aurite/aurite-agents/docs/notebooks/config. No project-specific components will be loaded.
WARNING  [aurite.config.project_manager] Project file not found at /home/wilcoxr/workspace/aurite/aurite-agents/docs/notebooks/aurite_config.json. Creating an empty project named 'aurite_config.json'.
INFO     [aurite.host.host] MCP Host initialization attempt finished. Successfully initialized 0/0 configured clients. 
INFO     [aurite.host_manager] Aurite initialization complete.


✅ Aurite initialized successfully!


## Section 2: Create the GadgetCo Knowledge Base

Let's create the company files that your support bot will reference. These files represent the internal knowledge base that a real customer support agent would have access to.

In [5]:
# Create the FAQ document
faq_content = """# GadgetCo Smart-Widget FAQ

## General Questions

**Q: What is the Smart-Widget?**
A: The Smart-Widget is GadgetCo's flagship IoT device that combines home automation, health monitoring, and entertainment features in one sleek package.

**Q: How do I set up my Smart-Widget?**
A: Please refer to the setup guide in the Smart-Widget manual. The basic steps are: 1) Download the GadgetCo app, 2) Create an account, 3) Follow the in-app setup wizard.

**Q: What's the warranty period?**
A: All Smart-Widget devices come with a 2-year manufacturer warranty covering defects and malfunctions under normal use.

**Q: Can I return my Smart-Widget?**
A: Yes! We offer a 30-day return policy for unopened items and a 14-day return policy for opened items in original condition.

## Troubleshooting

**Q: My Smart-Widget won't connect to WiFi. What should I do?**
A: Try these steps: 1) Ensure your WiFi password is correct, 2) Move the device closer to your router, 3) Restart both the device and your router, 4) Check if your network supports 2.4GHz (required for Smart-Widget).

**Q: The device isn't responding to voice commands.**
A: Check that: 1) The microphone isn't muted (blue light should be on), 2) You're within 10 feet of the device, 3) There's no loud background noise, 4) Voice recognition is enabled in the app settings.

**Q: My Smart-Widget keeps disconnecting from the app.**
A: This usually indicates a network issue. Try: 1) Checking your internet connection, 2) Restarting the Smart-Widget, 3) Updating the GadgetCo app, 4) Re-pairing the device in the app.

## Technical Specifications

**Q: What are the Smart-Widget's technical specs?**
A: Please refer to the technical specifications section in the Smart-Widget manual for detailed information.

**Q: Is the Smart-Widget compatible with other smart home systems?**
A: Yes! The Smart-Widget supports integration with Amazon Alexa, Google Assistant, and Apple HomeKit. Setup instructions are in the manual.
"""

faq_path = docs_dir / "faq.md"
with open(faq_path, "w") as f:
    f.write(faq_content)

print(f"✅ Created {faq_path}")
print(f"   - {len(faq_content)} characters")
print(f"   - Contains answers to common customer questions")


✅ Created customer_support_docs/faq.md
   - 1973 characters
   - Contains answers to common customer questions


In [6]:
# Create the manuals directory and Smart-Widget manual
manuals_dir = docs_dir / "manuals"
manuals_dir.mkdir(exist_ok=True)

manual_content = """# Smart-Widget User Manual

## Table of Contents
1. [Getting Started](#getting-started)
2. [Setup Instructions](#setup-instructions)
3. [Features](#features)
4. [Technical Specifications](#technical-specifications)
5. [Troubleshooting](#troubleshooting)
6. [Smart Home Integration](#smart-home-integration)

## Getting Started

Congratulations on your purchase of the GadgetCo Smart-Widget! This manual will guide you through setup and usage.

### What's in the Box
- Smart-Widget device
- Power adapter
- Quick start guide
- Warranty card

### Initial Setup Requirements
- Smartphone or tablet (iOS 12+ or Android 8+)
- WiFi network (2.4GHz required)
- GadgetCo account (free)

## Setup Instructions

### Step 1: Download the App
Download the "GadgetCo Home" app from the App Store or Google Play Store.

### Step 2: Create Account
Open the app and create your free GadgetCo account using your email address.

### Step 3: Power On Device
Connect the Smart-Widget to power using the included adapter. The device will show a pulsing blue light when ready for setup.

### Step 4: Add Device
In the app, tap "Add Device" and follow the setup wizard. The app will guide you through WiFi connection and device pairing.

### Step 5: Customize Settings
Once connected, customize your device name, location, and preferences in the app.

## Features

### Voice Control
The Smart-Widget responds to the wake word "Hey Widget" followed by your command. Supported commands include:
- "What's the weather?"
- "Set a timer for 10 minutes"
- "Play my morning playlist"
- "Turn on the living room lights"

### Health Monitoring
The device can track:
- Room temperature and humidity
- Air quality index
- Sleep patterns (when placed in bedroom)
- Activity reminders

### Entertainment
- Stream music from Spotify, Apple Music, or Amazon Music
- Built-in speakers with 360-degree sound
- Podcast and audiobook support
- Multi-room audio (with multiple devices)

### Home Automation
Control compatible smart home devices:
- Smart lights and switches
- Thermostats
- Security cameras
- Door locks

## Technical Specifications

### Hardware
- Processor: ARM Cortex-A53 quad-core
- RAM: 1GB DDR3
- Storage: 8GB eMMC
- WiFi: 802.11 b/g/n (2.4GHz)
- Bluetooth: 5.0
- Speakers: Dual 10W drivers
- Microphones: 4-array far-field

### Sensors
- Temperature sensor (±0.5°C accuracy)
- Humidity sensor (±3% accuracy)
- Air quality sensor (PM2.5, VOC)
- Ambient light sensor
- Motion detector (PIR)

### Physical
- Dimensions: 6.5\" × 6.5\" × 3.2\"
- Weight: 2.1 lbs
- Power: 12V DC, 2A
- Operating temperature: 32°F to 95°F
- Humidity range: 20% to 80% RH

## Troubleshooting

### Device Won't Power On
1. Check power adapter connection
2. Try a different power outlet
3. Ensure power adapter is the original GadgetCo adapter
4. Contact support if issue persists

### WiFi Connection Issues
1. Verify WiFi password is correct
2. Ensure network is 2.4GHz (5GHz not supported)
3. Move device closer to router during setup
4. Restart router and try again
5. Check for network firewall blocking device

### Voice Commands Not Working
1. Check microphone isn't muted (should show blue light)
2. Speak clearly from within 10 feet
3. Reduce background noise
4. Retrain voice recognition in app settings
5. Update device firmware through app

### App Connection Problems
1. Ensure phone and device are on same WiFi network
2. Update GadgetCo Home app to latest version
3. Restart the app
4. Re-pair device if necessary
5. Check internet connection stability

### Factory Reset
If other troubleshooting fails:
1. Hold the reset button (small button on bottom) for 10 seconds
2. Device will flash red, then return to setup mode (pulsing blue)
3. Re-add device through app setup process

## Smart Home Integration

### Amazon Alexa
1. Open Alexa app
2. Go to Skills & Games
3. Search for \"GadgetCo\"
4. Enable skill and link your GadgetCo account
5. Discover devices

### Google Assistant
1. Open Google Home app
2. Tap \"+\" then \"Set up device\"
3. Choose \"Works with Google\"
4. Search for \"GadgetCo\" and link account
5. Assign devices to rooms

### Apple HomeKit
1. Open Home app on iOS device
2. Tap \"+\" then \"Add Accessory\"
3. Scan QR code on device bottom or enter setup code
4. Follow setup prompts
5. Assign to room and configure scenes

## Support

For additional help:
- Visit support.gadgetco.com
- Email: support@gadgetco.com
- Phone: 1-800-GADGET-1
- Live chat available 24/7 through the app

Warranty and return information available at gadgetco.com/warranty
"""

manual_path = manuals_dir / "smart_widget_manual.txt"
with open(manual_path, "w") as f:
    f.write(manual_content)

print(f"✅ Created {manual_path}")
print(f"   - {len(manual_content)} characters")
print(f"   - Contains detailed technical information and troubleshooting")

print("\\n📁 Knowledge base files created:")
print(f"   - {faq_path.relative_to(docs_dir)}")
print(f"   - {manual_path.relative_to(docs_dir)}")


✅ Created customer_support_docs/manuals/smart_widget_manual.txt
   - 4526 characters
   - Contains detailed technical information and troubleshooting
\n📁 Knowledge base files created:
   - faq.md
   - manuals/smart_widget_manual.txt


## Section 3: Understanding the Tools You'll Need

For this project, you'll need to use two types of tools:

1. **Local File Storage** - To read the company knowledge base files
2. **External Search** - To find information not in the internal documents

Let's explore these tools and understand how they work.

In [7]:
# Let's register a simple agent to test the local file storage tools
from aurite.config.config_models import AgentConfig

test_agent_config = AgentConfig(
    name="File Test Agent",
    system_prompt="You are a helpful assistant that can read and list files. Use the available tools to help users understand what files are available and read their contents.",
    mcp_servers=["local_file_storage"]
)

await aurite.register_agent(test_agent_config)
print("✅ Registered File Test Agent")

INFO     [aurite.host.host] Attempting to dynamically register client: local_file_storage
INFO     [aurite.host.host] Client 'local_file_storage' dynamically registered and initialized successfully.


✅ Registered File Test Agent


In [8]:
# Test the file listing capability
query = "Can you list what files are available in this directory?"

result = await aurite.run_agent(
    agent_name="File Test Agent",
    user_message=query
)

display_agent_response("File Test Agent", query, result.primary_text)

WARNING  [aurite.execution.facade] Facade: Agent 'File Test Agent' running without a specific LLMConfig ID. Creating temporary, non-cached client using default OpenAI config.
INFO     [aurite.components.llm.providers.openai_client] OpenAIClient initialized for model gpt-3.5-turbo using direct API calls.
INFO     [aurite.execution.facade] Facade: Running conversation for Aurite Agent 'File Test Agent'...
INFO     [aurite.host.host] Executing tool 'list_directory' on client 'local_file_storage' for agent 'File Test Agent'
INFO     [aurite.execution.facade] Facade: Aurite Agent 'File Test Agent' conversation finished.



  <div style="border: 1px solid #D1D5DB; border-radius: 8px; margin-top: 20px; font-family: sans-serif; box-shadow: 0 4px 6px rgba(0,0,0,0.05);">
    <div style="background-color: #F3F4F6; padding: 10px 15px; border-bottom: 1px solid #D1D5DB; border-radius: 8px 8px 0 0;">
      <h3 style="margin: 0; font-size: 16px; color: #1F2937; display: flex; align-items: center;">
        <span style="margin-right: 8px;">🤖</span>
        Agent Response: <code style="background-color: #E5E7EB; color: #374151; padding: 2px 6px; border-radius: 4px; margin-left: 8px;">File Test Agent</code>
      </h3>
    </div>
    <div style="padding: 15px;">
      <p style="margin: 0 0 10px 0; color: #6B7280; font-size: 14px;">
        <strong>Your Query:</strong>
      </p>
      <p style="background-color: #F9FAFB; margin: 0 0 15px 0; color: #1F2937; border: 1px solid #E5E7EB; border-left: 3px solid #9CA3AF; padding: 10px 12px; border-radius: 4px;">
        <em>"Can you list what files are available in this directory?"</em>
      </p>
      <hr style="border: none; border-top: 1px dashed #D1D5DB; margin-bottom: 15px;">
      <p style="margin: 0 0 10px 0; color: #6B7280; font-size: 14px;">
        <strong>Result:</strong>
      </p>
      <div style="background-color: #FFFFFF; padding: 15px; border-radius: 5px; border: 1px solid #E5E7EB; color: #1F2937; font-size: 15px; line-height: 1.6;">
        The following files are available in this directory:
1. faq.md
2. manuals/smart_widget_manual.txt
      </div>
    </div>
  </div>
  

In [9]:
# Test reading the FAQ file
query = "Can you read the faq.md file and tell me what topics it covers?"

result = await aurite.run_agent(
    agent_name="File Test Agent",
    user_message=query
)

display_agent_response("File Test Agent", query, result.primary_text)

WARNING  [aurite.execution.facade] Facade: Agent 'File Test Agent' running without a specific LLMConfig ID. Creating temporary, non-cached client using default OpenAI config.
INFO     [aurite.components.llm.providers.openai_client] OpenAIClient initialized for model gpt-3.5-turbo using direct API calls.
INFO     [aurite.execution.facade] Facade: Running conversation for Aurite Agent 'File Test Agent'...
INFO     [aurite.host.host] Executing tool 'read_file' on client 'local_file_storage' for agent 'File Test Agent'
INFO     [aurite.execution.facade] Facade: Aurite Agent 'File Test Agent' conversation finished.



  <div style="border: 1px solid #D1D5DB; border-radius: 8px; margin-top: 20px; font-family: sans-serif; box-shadow: 0 4px 6px rgba(0,0,0,0.05);">
    <div style="background-color: #F3F4F6; padding: 10px 15px; border-bottom: 1px solid #D1D5DB; border-radius: 8px 8px 0 0;">
      <h3 style="margin: 0; font-size: 16px; color: #1F2937; display: flex; align-items: center;">
        <span style="margin-right: 8px;">🤖</span>
        Agent Response: <code style="background-color: #E5E7EB; color: #374151; padding: 2px 6px; border-radius: 4px; margin-left: 8px;">File Test Agent</code>
      </h3>
    </div>
    <div style="padding: 15px;">
      <p style="margin: 0 0 10px 0; color: #6B7280; font-size: 14px;">
        <strong>Your Query:</strong>
      </p>
      <p style="background-color: #F9FAFB; margin: 0 0 15px 0; color: #1F2937; border: 1px solid #E5E7EB; border-left: 3px solid #9CA3AF; padding: 10px 12px; border-radius: 4px;">
        <em>"Can you read the faq.md file and tell me what topics it covers?"</em>
      </p>
      <hr style="border: none; border-top: 1px dashed #D1D5DB; margin-bottom: 15px;">
      <p style="margin: 0 0 10px 0; color: #6B7280; font-size: 14px;">
        <strong>Result:</strong>
      </p>
      <div style="background-color: #FFFFFF; padding: 15px; border-radius: 5px; border: 1px solid #E5E7EB; color: #1F2937; font-size: 15px; line-height: 1.6;">
        The faq.md file covers the following topics:

- General Questions:
  - Description of the Smart-Widget
  - Setup instructions
  - Warranty information
  - Return policy

- Troubleshooting:
  - Connectivity issues with WiFi
  - Voice command troubleshooting
  - App disconnecting from the device

- Technical Specifications:
  - Details about the Smart-Widget's technical specifications
  - Compatibility with other smart home systems
      </div>
    </div>
  </div>
  

## Section 4: Your Challenge - Build Chip the Customer Support Agent

Now it's your turn! You need to create "Chip", a customer support agent for GadgetCo. Here's what Chip should do:

### Chip's Requirements:

1. **Personality**: Friendly, patient, and professional customer support representative
2. **Process**: Follow a specific order when helping customers:
   - First check the FAQ file (`faq.md`)
   - If not found there, check the manual (`manuals/smart_widget_manual.txt`)
   - Only use external search if the answer isn't in internal documents
   - If no answer is found anywhere, escalate to human support
3. **Tools**: Use both `local_file_storage` and `duckduckgo_search`
4. **Rules**: Never guess, always cite sources, be helpful but honest

### Your Task:

In the cell below, create and register your "Chip" agent. Think carefully about:
- The system prompt (personality, process, rules)
- Which MCP servers to include
- How to instruct the agent to use tools in the right order

In [10]:
# YOUR CODE HERE: Create and register Chip the customer support agent

# Hint: You'll need to:
# 1. Create an AgentConfig with:
#    - name: "Chip"
#    - system_prompt: (your detailed instructions)
#    - mcp_servers: ["local_file_storage", "duckduckgo_search"]
# 2. Register the agent with aurite.register_agent()

# Example structure:
# chip_config = AgentConfig(
#     name="Chip",
#     system_prompt="""Your detailed system prompt here...""",
#     mcp_servers=["local_file_storage", "duckduckgo_search"]
# )
# await aurite.register_agent(chip_config)

print("TODO: Implement Chip the customer support agent!")

TODO: Implement Chip the customer support agent!


## Section 5: Test Your Agent

Once you've created Chip, test it with these scenarios to see how well it follows your instructions.

In [11]:
# Set up a dedicated directory for our knowledge base files
# and configure the file storage server to use it.
import os
from pathlib import Path

# Create a dedicated directory for the knowledge base
docs_dir = Path("customer_support_docs")
docs_dir.mkdir(exist_ok=True)

# Set the environment variable to this new directory.
# The local_file_storage server will use this path as its root.
os.environ['AURITE_PROJECT_DIR'] = str(docs_dir.resolve())

print(f"✅ Set AURITE_PROJECT_DIR to: {os.environ['AURITE_PROJECT_DIR']}")

✅ Set AURITE_PROJECT_DIR to: /home/wilcoxr/workspace/aurite/aurite-agents/docs/notebooks/customer_support_docs


In [12]:
# Initialize Aurite
from aurite import Aurite

aurite = Aurite()
await aurite.initialize()
print("✅ Aurite initialized successfully!")

INFO     [aurite.config.component_manager] User project config directory not found at /home/wilcoxr/workspace/aurite/aurite-agents/docs/notebooks/config. No project-specific components will be loaded.
WARNING  [aurite.config.project_manager] Project file not found at /home/wilcoxr/workspace/aurite/aurite-agents/docs/notebooks/aurite_config.json. Creating an empty project named 'aurite_config.json'.
INFO     [aurite.host.host] MCP Host initialization attempt finished. Successfully initialized 0/0 configured clients. 
INFO     [aurite.host_manager] Aurite initialization complete.


✅ Aurite initialized successfully!


In [13]:
# Test 3: Troubleshooting question
query = "My Smart-Widget won't connect to WiFi. I've tried entering the password multiple times but it keeps failing. What should I do?"

# Uncomment these lines after you've created Chip:
# result = await aurite.run_agent(
#     agent_name="Chip",
#     user_message=query
# )
# display_agent_response("Chip", query, result.primary_text)

print("Test 3: Troubleshooting - Create Chip first!")

Test 3: Troubleshooting - Create Chip first!


In [14]:
# Test 4: Question that might require external search
query = "I heard there might be a new Smart-Widget model coming out soon. Can you tell me about any upcoming product releases?"

# Uncomment these lines after you've created Chip:
# result = await aurite.run_agent(
#     agent_name="Chip",
#     user_message=query
# )
# display_agent_response("Chip", query, result.primary_text)

print("Test 4: External Search - Create Chip first!")

Test 4: External Search - Create Chip first!


## Section 6: Evaluation and Improvement

After testing your agent, consider these questions:

### Evaluation Criteria:

1. **Process Adherence**: Did Chip follow the prescribed process of checking FAQ → Manual → External Search?
2. **Response Quality**: Were the responses helpful, accurate, and appropriately detailed?
3. **Personality**: Did Chip maintain a consistent, professional, and friendly tone?
4. **Source Citation**: Did Chip properly cite where information came from?
5. **Tool Usage**: Did Chip use the file storage tools effectively?

### Common Challenges:

- **Tool Order**: Ensuring the agent checks internal documents before external search
- **Source Attribution**: Making sure the agent cites where information came from
- **Escalation**: Knowing when to admit it doesn't know something
- **Personality Consistency**: Maintaining the right tone throughout the conversation

### Improvement Ideas:

- Add more specific instructions about when to escalate
- Include example phrases for different situations
- Add empathy responses for frustrated customers
- Create more detailed troubleshooting workflows
- Add proactive suggestions ("While I have you, did you know...")

### Real-World Applications:

This customer support bot pattern can be adapted for various industries:
- **E-commerce**: Product information, order status, returns
- **SaaS**: Feature explanations, troubleshooting, account management
- **Healthcare**: Appointment scheduling, basic health information, insurance questions
- **Financial Services**: Account inquiries, transaction help, policy information

## Congratulations!

You've successfully learned how to build a multi-tool customer support agent using the Aurite framework. The key concepts you've mastered include:

- **System Prompt Design**: Creating detailed instructions for agent behavior
- **Multi-Tool Integration**: Using multiple MCP servers in a single agent
- **Conditional Logic**: Designing agents that follow specific processes
- **Knowledge Base Management**: Working with local file storage for company information
- **Fallback Strategies**: Implementing escalation when information isn't available

These skills form the foundation for building sophisticated AI agents that can handle real-world business scenarios!